 ## Explore data

In [2]:
import pandas as pd
import re
import spacy

# Caricare il modello di spaCy per l'analisi sintattica e semantica
nlp = spacy.load("en_core_web_sm")

# Definire le relazioni esistenti nel dataset
RELATIONS = [
    "Cause-Effect", "Instrument-Agency", "Product-Producer", "Content-Container",
    "Entity-Origin", "Entity-Destination", "Component-Whole", "Member-Collection",
    "Message-Topic", "Other"
]

def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for i in range(0, len(lines), 2):  # Ogni esempio è su 2 righe (frase + relazione)
        if '"' in lines[i]:  
            sentence = re.findall(r'"(.*?)"', lines[i])[0]
            
            # Estrai la relazione senza la parte con le parentesi
            relation = re.sub(r'\(.*?\)', '', lines[i + 1]).strip() if i + 1 < len(lines) else "Other"
            
            # Estrai le entità
            entities = re.findall(r'<e1>(.*?)</e1>', lines[i]) + re.findall(r'<e2>(.*?)</e2>', lines[i])
            data.append({'sentence': sentence, 'entities': entities, 'relation': relation})

    return pd.DataFrame(data)

# Caricare i dati
train_df = load_data('data/TRAIN_FILE.TXT')

# Mostra le prime righe per verifica
print(train_df.head())


                                            sentence  \
0  The system as described above has its greatest...   
1  The <e1>child</e1> was carefully wrapped and b...   
2  The <e1>author</e1> of a keygen uses a <e2>dis...   
3  A misty <e1>ridge</e1> uprises from the <e2>su...   
4  The <e1>student</e1> <e2>association</e2> is t...   

                    entities           relation  
0  [configuration, elements]    Component-Whole  
1            [child, cradle]              Other  
2     [author, disassembler]  Instrument-Agency  
3             [ridge, surge]              Other  
4     [student, association]  Member-Collection  


# Rule based approach

### l'idea che ho avuto è di prendere il verbo e di controllarne il prefisso

In [3]:
import spacy

# Caricare il modello di spaCy per il POS tagging
nlp = spacy.load("en_core_web_sm")

# Lista di prefissi e le loro corrispondenti relazioni
prefix_to_relation = {
    "cau": "Cause-Effect",  # Cause, causare
    "lead": "Cause-Effect",  # Lead
    "resu": "Cause-Effect",  # Result
    "use": "Instrument-Agency",  # Use
    "oper": "Instrument-Agency",  # Operate
    "make": "Product-Producer",  # Make
    "prod": "Product-Producer",  # Produce, Produce
    "manu": "Product-Producer",  # Manufacture
    "crea": "Product-Producer",  # Create
    "cont": "Content-Container",  # Contain
    "insi": "Content-Container",  # Inside
    "orig": "Entity-Origin",  # Originate
    "from": "Entity-Origin",  # From
    "move": "Entity-Destination",  # Move
    "to": "Entity-Destination",  # To
    "part": "Component-Whole",  # Part
    "incl": "Component-Whole",  # Include
    "belon": "Member-Collection",  # Belong
    "membe": "Member-Collection",  # Member of
    "talk": "Message-Topic",  # Talk
    "topi": "Message-Topic",  # Topic
}

def classify_relation_using_verb_with_prefixes(sentence, entities):
    # Controlla se entities contiene esattamente due elementi
    if len(entities) != 2:
        return "Other"  # Se non ci sono esattamente due entità, ritorna "Other"

    # Analizzare la frase con spaCy
    doc = nlp(sentence)

    # Estrarre e1 ed e2 dalle entità
    e1, e2 = entities

    # Trova il verbo tra e1 ed e2
    verb = None
    for token in doc:
        if token.pos_ == "VERB":
            verb = token.lemma_  # Usare la forma lemmatizzata del verbo
            break  # Prendiamo il primo verbo che troviamo tra e1 ed e2

    # Se c'è un verbo, controlliamo i prefissi
    if verb:
        verb_prefix = verb[:4]  # Prendi le prime 4 lettere del verbo
        if verb_prefix in prefix_to_relation:
            return prefix_to_relation[verb_prefix]  # Restituisci la relazione basata sul prefisso

    return "Other"  # Se nessun prefisso corrisponde

# Applicare la funzione al training set
train_df['predicted_relation'] = train_df.apply(lambda row: classify_relation_using_verb_with_prefixes(row['sentence'], row['entities']), axis=1)

# Visualizza i risultati
print(train_df[['sentence', 'entities', 'predicted_relation']].head())


                                            sentence  \
0  The system as described above has its greatest...   
1  The <e1>child</e1> was carefully wrapped and b...   
2  The <e1>author</e1> of a keygen uses a <e2>dis...   
3  A misty <e1>ridge</e1> uprises from the <e2>su...   
4  The <e1>student</e1> <e2>association</e2> is t...   

                    entities predicted_relation  
0  [configuration, elements]              Other  
1            [child, cradle]              Other  
2     [author, disassembler]  Instrument-Agency  
3             [ridge, surge]              Other  
4     [student, association]              Other  


In [4]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Calcolare l'accuratezza
accuracy = accuracy_score(train_df['relation'], train_df['predicted_relation'])
print(f"Accuracy: {accuracy * 100:.2f}%")

# Calcolare Precision, Recall e F1-Score per ciascuna classe
precision, recall, f1, support = precision_recall_fscore_support(
    train_df['relation'], train_df['predicted_relation'], average=None, labels=train_df['relation'].unique()
)

# Visualizzare i risultati delle metriche per ogni classe
metrics_df = pd.DataFrame({
    'Class': train_df['relation'].unique(),
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print(metrics_df)

# Calcolare la F1-Score macro-media
macro_f1 = f1.mean()
print(f"Macro F1-Score: {macro_f1:.2f}")

# Calcolare la F1-Score pesata
weighted_f1 = (precision * support).sum() / support.sum()
print(f"Weighted F1-Score: {weighted_f1:.2f}")


Accuracy: 19.39%
                 Class  Precision    Recall  F1-Score  Support
0      Component-Whole   0.296875  0.020191  0.037811      941
1                Other   0.176502  0.929078  0.296649     1410
2    Instrument-Agency   0.354839  0.087302  0.140127      504
3    Member-Collection   0.000000  0.000000  0.000000      690
4         Cause-Effect   0.642857  0.026919  0.051675     1003
5   Entity-Destination   0.651163  0.033136  0.063063      845
6    Content-Container   0.426357  0.101852  0.164425      540
7        Message-Topic   0.300000  0.004732  0.009317      634
8     Product-Producer   0.285714  0.108787  0.157576      717
9        Entity-Origin   0.909091  0.013966  0.027510      716
10                       0.000000  0.000000  0.000000      118
Macro F1-Score: 0.09
Weighted F1-Score: 0.39


/home/mf/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Feature vector + machine learning solution based

### Creazione del feature vector, bool features controllando determinate keywords

In [10]:
import spacy
import pandas as pd
import re

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the set of all possible POS tags in spaCy
possible_pos_tags = [
    "ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART",
    "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"
]

# List of individual keywords (lemmatized)
keywords = [
    "cause", "lead", "result", "use", "operate", "make", "produce", "manufacture", "create",
    "contain", "inside", "originate", "from", "move", "to", "part", "include",
    "belong", "member", "talk", "topic"
]

# Function to extract features
def extract_features(sentence, entities):
    # Check if exactly two entities exist
    if len(entities) != 2:
        return None  

    # Remove entity tags from the sentence
    sentence_cleaned = re.sub(r'<e1>|</e1>|<e2>|</e2>', '', sentence)

    # Process the sentence with spaCy
    doc = nlp(sentence_cleaned)

    # Extract entity texts
    e1, e2 = entities

    # Initialize feature vector
    feature_vector = {"sentence_length": len(sentence_cleaned.split())}

    # Initialize one-hot encoding for POS tags (all 0 by default)
    for pos in possible_pos_tags:
        feature_vector[f"e1_is_{pos}"] = 0
        feature_vector[f"e2_is_{pos}"] = 0

    # Find the POS tags for e1 and e2
    for token in doc:
        if token.text == e1:
            if token.pos_ in possible_pos_tags:
                feature_vector[f"e1_is_{token.pos_}"] = 1
        elif token.text == e2:
            if token.pos_ in possible_pos_tags:
                feature_vector[f"e2_is_{token.pos_}"] = 1

    # Create binary features for each keyword
    lemmas_in_sentence = {token.lemma_ for token in doc}  
    for keyword in keywords:
        feature_vector[f"has_{keyword}"] = 1 if keyword in lemmas_in_sentence else 0

    return feature_vector

# Apply the function to the training set
train_features = []
for _, row in train_df.iterrows():
    sentence = row['sentence']
    entities = row['entities']
    feature_vector = extract_features(sentence, entities)
    
    if feature_vector:
        feature_vector['actual_relation'] = row['relation']
        train_features.append(feature_vector)

# Convert to a DataFrame
train_feature_df = pd.DataFrame(train_features)

# Display the first 5 feature vectors
print(train_feature_df.head())
print(train_feature_df.size)





   sentence_length  e1_is_ADJ  e2_is_ADJ  e1_is_ADP  e2_is_ADP  e1_is_ADV  \
0               27          0          0          0          0          0   
1                6          0          0          0          0          0   
2               24          0          0          0          0          0   
3               14          0          0          0          0          0   
4               13          0          0          0          0          0   

   e2_is_ADV  e1_is_AUX  e2_is_AUX  e1_is_CCONJ  ...  has_from  has_move  \
0          0          0          0            0  ...         0         0   
1          0          0          0            0  ...         0         0   
2          0          0          0            0  ...         0         0   
3          0          0          0            0  ...         0         0   
4          0          0          0            0  ...         0         0   

   has_to  has_part  has_include  has_belong  has_member  has_talk  has_topic  \

 ### SVM

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assicurati che la colonna di relazione (target) sia separata
X = train_feature_df.drop(columns=['actual_relation'])  # Le caratteristiche
y = train_feature_df['actual_relation']  # Le etichette (relazioni effettive)

# Suddividere i dati in training e validation set (80% / 20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Creare il modello SVM
svm_model = SVC(kernel='linear', random_state=42)

# Allenare il modello
svm_model.fit(X_train, y_train)

# Predire le etichette sul set di validazione
y_pred = svm_model.predict(X_val)

# Calcolare l'accuratezza
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Matrice di confusione
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Report di classificazione
print("\nClassification Report:")
print(classification_report(y_val, y_pred))



Accuracy: 0.3300
Confusion Matrix:
[[ 88   4   0   6  33   0   1   0  17   8]
 [  4  13   6  13  10   7   2   1  91   0]
 [  1   1  21  11   3   0   0   0  32   0]
 [  3   4   3  80   7   3   0   0  41   1]
 [  2   1   2   6  70   2   0   0  25   1]
 [  1   2   0  11   6  22   0   0  32   0]
 [  2   3   2  14  14   4   3   0  85   0]
 [  2   1   1  32  12   5   0   7  57   1]
 [  7   6   3  43  40  10   1   0 105   4]
 [  2   1   0   7  19   5   1   0  71  14]]

Classification Report:
                    precision    recall  f1-score   support

      Cause-Effect       0.79      0.56      0.65       157
   Component-Whole       0.36      0.09      0.14       147
 Content-Container       0.55      0.30      0.39        69
Entity-Destination       0.36      0.56      0.44       142
     Entity-Origin       0.33      0.64      0.43       109
 Instrument-Agency       0.38      0.30      0.33        74
 Member-Collection       0.38      0.02      0.04       127
     Message-Topic       0.88

# Neural Network

In [15]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.amp import autocast, GradScaler  # For mixed precision


# Load the DistilBERT tokenizer and model (smaller model)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Label encoding for relations
label_encoder = LabelEncoder()
train_df['relation_label'] = label_encoder.fit_transform(train_df['relation'])

# Split train_df into train (80%), val (10%), and test (10%)
train_df, test_val_df = train_test_split(train_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

# Custom Dataset for BERT tokenization
class RelationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=32):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        sentence = row['sentence']
        relation_label = row['relation_label']

        # Tokenize the sentence
        encoding = self.tokenizer(
            sentence, 
            add_special_tokens=True, 
            truncation=True, 
            padding='max_length', 
            max_length=self.max_length, 
            return_attention_mask=True, 
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(relation_label, dtype=torch.long)
        }

# Instantiate Datasets and DataLoaders
train_dataset = RelationDataset(train_df, tokenizer)
val_dataset = RelationDataset(val_df, tokenizer)
test_dataset = RelationDataset(test_df, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

class SelfAttentionModel(nn.Module):
    def __init__(self, hidden_dim, num_labels):
        super(SelfAttentionModel, self).__init__()
        
        # Caricamento di un modello più piccolo (DistilBERT) per ridurre la memoria
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
        # Layer Fully Connected per la classificazione
        self.fc = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask):
        # Ottenere l'output di DistilBERT (last hidden state)
        distilbert_output = self.distilbert(input_ids, attention_mask=attention_mask)
        
        # Estrarre l'output dell'ultimo stato nascosto
        hidden_state = distilbert_output.last_hidden_state
        
        # Usare l'output del token [CLS] per la classificazione (primo token)
        cls_token_output = hidden_state[:, 0, :]  # Prendere il primo token (CLS)

        # Passare attraverso un layer fully connected per ottenere le predizioni
        logits = self.fc(cls_token_output)  # La forma ora è (batch_size, num_labels)
        
        return logits



# Model, Loss, Optimizer
model = SelfAttentionModel(hidden_dim=768, num_labels=len(label_encoder.classes_))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Mixed Precision Scaler
scaler = GradScaler()

# Training Loop
def train_epoch(model, dataloader, loss_fn, optimizer):
    model.train()
    total_loss = 0
    correct_predictions = 0

    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Forward pass with mixed precision
        optimizer.zero_grad()
        with autocast(device_type="cuda"):  # Use mixed precision
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
        
        total_loss += loss.item()

        # Backward pass with mixed precision
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Calculate accuracy
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == labels)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions.double() / len(dataloader.dataset)
    return avg_loss, accuracy

# Validation Loop
def eval_epoch(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0
    correct_predictions = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            # Calculate accuracy
            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions.double() / len(dataloader.dataset)
    return avg_loss, accuracy

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training and Evaluation
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Train for one epoch
    train_loss, train_accuracy = train_epoch(model, train_dataloader, loss_fn, optimizer)
    print(f"Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")
    
    # Evaluate on validation data
    val_loss, val_accuracy = eval_epoch(model, val_dataloader, loss_fn)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    
    # Evaluate on test data
    test_loss, test_accuracy = eval_epoch(model, test_dataloader, loss_fn)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")

    # Clear GPU memory after each epoch
    #torch.cuda.empty_cache()


Epoch 1/5
Training Loss: 1.9720, Accuracy: 0.3238
Validation Loss: 1.2996, Accuracy: 0.6325
Test Loss: 1.3806, Accuracy: 0.5766
Epoch 2/5
Training Loss: 1.0070, Accuracy: 0.6739
Validation Loss: 0.8197, Accuracy: 0.7229
Test Loss: 0.9844, Accuracy: 0.6486
Epoch 3/5
Training Loss: 0.6856, Accuracy: 0.7841
Validation Loss: 0.7382, Accuracy: 0.7530
Test Loss: 0.8632, Accuracy: 0.6877
Epoch 4/5
Training Loss: 0.4984, Accuracy: 0.8469
Validation Loss: 0.7046, Accuracy: 0.7470
Test Loss: 0.8593, Accuracy: 0.6817
Epoch 5/5
Training Loss: 0.3573, Accuracy: 0.9003
Validation Loss: 0.7446, Accuracy: 0.7319
Test Loss: 0.8976, Accuracy: 0.6847
